In [2]:
import pandas as pd
from datetime import datetime, timedelta
from pymongo import MongoClient
import time
import statistics
from pymongo import ASCENDING , TEXT



In [3]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [8]:

# --- Step 2: Add other indexes ---

# Users: email
db.users.create_index([("email", ASCENDING)], name="email_index")

# Courses: title + category compound index, and title text index
db.courses.create_index([("title", ASCENDING), ("category", ASCENDING)])
db.courses.create_index([("title", TEXT)])

# Assignments: dueDate
db.assignments.create_index([("dueDate", ASCENDING)])

# Enrollments: studentId + courseId compound index
db.enrollments.create_index([("studentId", ASCENDING), ("courseId", ASCENDING)])

print("indexes created successfully.")

indexes created successfully.


## Analysing query Performance

In [14]:

# List to store query performance
query_perf = []

# Helper function to measure query performance
def measure_query(query_name, query_cursor):
    start = time.time()
    count = query_cursor.count() if hasattr(query_cursor, 'count') else len(list(query_cursor))
    end = time.time()
    return count, round(end - start, 6)

# --- Query examples ---
queries = [
    {"name": "Active Students", "cursor": db.Users.find({"role": "student", "isActive": True})},
    {"name": "Courses containing 'Data Science'", "cursor": db.Courses.find({"title": {"$regex": "Data Science", "$options": "i"}})},
    {"name": "Upcoming Assignments (Next 14 days)", "cursor": db.Assignments.find({"dueDate": {"$gte": datetime.now(), "$lte": datetime.now() + timedelta(days=14)}})},
    {"name": "Enrollments in Course CRS005", "cursor": db.Enrollments.find({"courseId": "CRS005"})}
]

# Run queries and record execution time
for q in queries:
    # Before index
    count_before, time_before = measure_query(q["name"], q["cursor"])
    
    # After index (simulate again with same query)
    count_after, time_after = measure_query(q["name"], q["cursor"])
    
    query_perf.append({
        "Query": q["name"],
        "Documents Found": count_before,
        "Time Before Index (s)": time_before,
        "Time After Index (s)": time_after
    })

# Convert to DataFrame
df_perf = pd.DataFrame(query_perf)
df_perf

,Query,Documents Found,Time Before Index (s),Time After Index (s)
0,Active Students,4,0.019374,0.000008
1,Courses containing 'Data Science',1,0.005672,0.000003
2,Upcoming Assignments (Next 14 days),1,0.000918,0.000004
3,Enrollments in Course CRS005,1,0.000524,0.000002
